In [1]:
import pandas as pd
import numpy as np

In [2]:
abstract_path = "/home/yh/dzx/work/BioNLP/work_data/abstract_pubtator.txt"
text = open(abstract_path, "r").readlines()

展示pubtator结果基本文件格式：
一篇文章拆分为一行title，一行abstract，若干行提取的实体信息

In [3]:
print("tilte: ", text[0])
print("abstract: ", text[1])
print("oneof the entity: ", text[2])

tilte:  37796141|t|Current status of exercise and its impact on quality of life in patients undergoing peritoneal dialysis in the post-COVID-19 period.

abstract:  37796141|a|OBJECTIVES: The aims of this study were to investigate the current status and the influence factors of exercise, and to explore the impact of exercise on the quality of life (QoL) in peritoneal dialysis (PD) patients in the post-COVID-19 period. MATERIALS AND METHODS: Those PD patients who were followed up between September 2020 and August 2021 were enrolled. The collected data included demographic information, clinical data, exercise data, and QoL. RESULTS: In total, 339 PD patients were included in this cross-sectional study. The mean age was 44.0 +- 13.0 years, with a median PD duration of 6.7 (1.7 - 41.9) months. The primary renal disease was glomerulonephritis (68.4%). 277 (81.7%) PD patients performed exercise, with median exercise time 5.0 (3.5 - 7.8) hours per week. The main type of exercise was slow walki

获取pubtator结果时每一篇文章间以两行空行分隔，根据此分隔符将pubtator结果拆分为

In [4]:
#获取pubtator结果时每一篇文章间以两行空行分隔，将两行空行换成一行空行
for i in range(len(text)-1):
    if text[i] == "\n" and text[i+1] == "\n":
        text[i] = ""
#删除空元素
text = [x for x in text if x != ""]

In [5]:
#查看分隔结果
print(text[30:34])

['37796141\t1818\t1820\tPD\tDisease\tMESH:D010300\n', '37796141\t1821\t1829\tpatients\tSpecies\t9606\n', '\n', '37795527|t|Prolonged shedding of viable SARS-CoV-2 in immunocompromised patients with haematological malignancies: A prospective study.\n']


In [6]:
#获取'\n'编号，依此分隔成文章
index = []
for i in range(len(text)):
    if text[i] == "\n":
        index.append(i)
print("index: ", index)

index:  [32, 50, 65, 93, 133, 141, 153, 157, 173, 188, 193, 217, 232, 268, 274, 318, 331, 358, 388, 412, 451, 464, 475, 509, 514, 535, 539, 569, 591, 628, 654, 682, 699, 704, 732, 741, 760, 786, 815, 841, 868, 928, 943, 957, 976, 1009, 1028, 1068, 1072, 1079, 1100, 1105, 1125, 1149, 1179, 1213, 1227, 1256, 1288, 1338, 1376, 1405, 1420, 1444, 1457, 1482, 1513, 1534, 1556, 1579, 1584, 1603, 1607, 1622, 1645, 1649, 1653, 1657, 1678, 1725, 1730, 1756, 1786, 1808, 1838, 1867, 1881, 1900, 1923, 1945, 1960, 1977, 2004, 2034, 2063, 2091, 2118, 2138, 2155, 2189, 2212, 2224, 2251, 2270, 2296, 2336, 2371, 2378, 2402, 2435, 2462, 2479, 2519, 2534, 2546, 2582, 2586, 2602, 2626, 2655, 2691, 2708, 2721, 2736, 2740, 2767, 2793, 2819, 2848, 2867, 2878, 2908, 2946, 2974, 2981, 3022, 3052, 3083, 3095, 3122, 3134, 3162, 3184, 3211, 3215, 3238, 3243, 3258, 3262, 3313, 3320, 3341, 3363, 3375, 3390, 3426, 3459, 3463, 3479, 3518, 3528, 3539, 3559, 3580, 3603, 3615, 3643, 3669, 3674, 3703, 3716, 3727, 3762, 37

In [11]:
#根据'\n'的编号，依此分隔成文章
article = []
for i in range(len(index)):
    if i == 0:
        article.append(text[:index[i]])
    else:
        article.append(text[index[i-1]+1:index[i]])
#输出至abstract.txt
with open("/home/yh/dzx/work/BioNLP/work_data/abstract.txt", "w") as f:
    for i in range(len(article)):
        for j in range(len(article[i])):
            f.write(article[i][j])
    f.write("\n")

In [8]:
#将每个文章的实体部分提取出来
entity = []
for i in range(len(article)):
    entity.append(article[i][2:])
    
#删除每一个元素最后的换行符 以制表符分隔
for i in range(len(entity)):
    for j in range(len(entity[i])):
        entity[i][j] = entity[i][j].strip("\n").split("\t")
        
#每一个元素化作数据框拼接在一起
entity_df = []
for i in range(len(entity)):
    entity_df.append(pd.DataFrame(entity[i]))


In [9]:
#将所有数据合并成一个数据框输出成tsv文件,重命名列名
entity_df = pd.concat(entity_df)
entity_df.columns = ["PMID", "start", "end", "entity", "type", "ID"]
entity_df.to_csv("entity.tsv", sep="\t", index=False, header=False)

使用linux终端命令grep出mutation共有38行，针对每一行mutation：
1.提取出其所在的文章的title和abstract，汇总
2.提取出所有mutation的实体信息，汇总

In [10]:
#筛选出实体类型为含有Mutation字眼的实体
entity_df = entity_df[entity_df["type"].str.contains("Mutation")]
print(entity_df.head())

       PMID start  end entity             type  \
5  37722481   674  679  K129R  ProteinMutation   
6  37722481   681  686  V722I  ProteinMutation   
7  37722481   691  696  V987F  ProteinMutation   
8  37722481   826  831  E340K  ProteinMutation   
9  37722481   833  838  K356R  ProteinMutation   

                                                ID  
5  tmVar:p|SUB|K|129|R;HGVS:p.K129R;VariantGroup:3  
6  tmVar:p|SUB|V|722|I;HGVS:p.V722I;VariantGroup:5  
7  tmVar:p|SUB|V|987|F;HGVS:p.V987F;VariantGroup:4  
8  tmVar:p|SUB|E|340|K;HGVS:p.E340K;VariantGroup:6  
9  tmVar:p|SUB|K|356|R;HGVS:p.K356R;VariantGroup:0  


In [ ]:
entity_df.to_csv("../work_data/mutation_entity.tsv", sep="\t", index=False, header=False)